In [6]:
import os
import sys
import tensorflow as tf

ERROR! Session/line number was not unique in database. History logging moved to new session 163


In [7]:
DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult'
TRAINING_FILE = 'adult.data'
TRAINING_URL = '%s/%s' % (DATA_URL, TRAINING_FILE)
EVAL_FILE = 'adult.test'
EVAL_URL = '%s/%s' % (DATA_URL, EVAL_FILE)
_CSV_COLUMN_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
[0], [0], [0], [''], ['']]
_CSV_COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]

In [8]:
train_file = 'D:/data/adult.data'
eval_file = 'D:/data/adult.test'

In [9]:
dataset = tf.data.TextLineDataset(eval_file)

In [10]:
print(dataset)

<TextLineDataset shapes: (), types: tf.string>


In [11]:
iterator1 = dataset.make_one_shot_iterator()
x1 = iterator1.get_next()
with tf.Session() as sess:
    #for i in range(4):
    print(sess.run(x1))

b'25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K'


In [12]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have run census_dataset.py and '
      'set the --data_dir argument to the correct path.' % data_file)

  def parse_csv(value):
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('income_bracket')
    classes = tf.equal(labels, '>50K')  # binary classification
    return features, classes

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)
  return dataset

In [13]:
def build_model_columns():
  """Builds a set of wide and deep feature columns."""
  # Continuous variable columns
  age = tf.feature_column.numeric_column('age')
  education_num = tf.feature_column.numeric_column('education_num')
  capital_gain = tf.feature_column.numeric_column('capital_gain')
  capital_loss = tf.feature_column.numeric_column('capital_loss')
  hours_per_week = tf.feature_column.numeric_column('hours_per_week')

  education = tf.feature_column.categorical_column_with_vocabulary_list(
      'education', [
          'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
          'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
          '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

  marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
      'marital_status', [
          'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
          'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

  relationship = tf.feature_column.categorical_column_with_vocabulary_list(
      'relationship', [
          'Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried',
          'Other-relative'])

  workclass = tf.feature_column.categorical_column_with_vocabulary_list(
      'workclass', [
          'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
          'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

  # To show an example of hashing:
  occupation = tf.feature_column.categorical_column_with_hash_bucket(
      'occupation', hash_bucket_size=32)

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(
      age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

  # Wide columns and deep columns.
  base_columns = [
      education, marital_status, relationship, workclass, occupation,
      age_buckets,
  ]

  crossed_columns = [
      tf.feature_column.crossed_column(
          ['education', 'occupation'], hash_bucket_size=32),
      tf.feature_column.crossed_column(
          [age_buckets, 'education', 'occupation'],
          hash_bucket_size=32),
  ]

  wide_columns = base_columns + crossed_columns

  deep_columns = [
      age,
      education_num,
      capital_gain,
      capital_loss,
      hours_per_week,
      tf.feature_column.indicator_column(workclass),
      tf.feature_column.indicator_column(education),
      tf.feature_column.indicator_column(marital_status),
      tf.feature_column.indicator_column(relationship),
      # To show an example of embedding
      tf.feature_column.embedding_column(occupation, dimension=8),
  ]

  return deep_columns

In [14]:
deep_columns=build_model_columns()

In [15]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=deep_columns,
    hidden_units=[10, 10],
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Kaufman\\AppData\\Local\\Temp\\tmpz0r1zx9n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001707DED55F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
classifier.train(
input_fn=lambda:input_fn(train_file,10,False,32),
steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Kaufman\AppData\Local\Temp\tmpz0r1zx9n\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into C:\Users\Kaufman\AppData\Local\Temp\tmpz0r1zx9n\model.ckpt.
INFO:tensorflow:loss = 13.031178, step = 1001
INFO:tensorflow:global_step/sec: 521.13
INFO:tensorflow:loss = 9.766506, step = 1101 (0.193 sec)
INFO:tensorflow:global_step/sec: 901.419
INFO:tensorflow:loss = 7.340595, step = 1201 (0.111 sec)
INFO:tensorflow:global_step/sec: 877.693
INFO:tensorflow:loss = 6.1878996, step = 1301 (0.114 sec)
INFO:tensorflow:global_step/sec: 877.693
INFO:tensorflow:loss = 15.229997, step = 1401 (0.114 sec)
INFO:tensorflow:global_step/sec: 901.417
INFO:tensorflow:loss = 25.375515, step = 1501 (0.111 sec)
INFO:tenso

INFO:tensorflow:loss = 9.805748, step = 8901 (0.160 sec)
INFO:tensorflow:global_step/sec: 781.698
INFO:tensorflow:loss = 12.983517, step = 9001 (0.123 sec)
INFO:tensorflow:global_step/sec: 926.459
INFO:tensorflow:loss = 10.431665, step = 9101 (0.108 sec)
INFO:tensorflow:global_step/sec: 943.932
INFO:tensorflow:loss = 16.197224, step = 9201 (0.106 sec)
INFO:tensorflow:global_step/sec: 943.942
INFO:tensorflow:loss = 7.9439125, step = 9301 (0.106 sec)
INFO:tensorflow:global_step/sec: 840.817
INFO:tensorflow:loss = 7.3609705, step = 9401 (0.121 sec)
INFO:tensorflow:global_step/sec: 769.671
INFO:tensorflow:loss = 8.012523, step = 9501 (0.129 sec)
INFO:tensorflow:global_step/sec: 787.852
INFO:tensorflow:loss = 14.100448, step = 9601 (0.126 sec)
INFO:tensorflow:global_step/sec: 926.455
INFO:tensorflow:loss = 8.767084, step = 9701 (0.108 sec)
INFO:tensorflow:global_step/sec: 917.96
INFO:tensorflow:loss = 20.028183, step = 9801 (0.110 sec)
INFO:tensorflow:global_step/sec: 826.922
INFO:tensorflo

In [19]:
results = classifier.evaluate(input_fn=lambda:input_fn(eval_file,1,False,32))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-13-15:44:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Kaufman\AppData\Local\Temp\tmpz0r1zx9n\model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-13-15:44:36
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.84294575, accuracy_baseline = 0.76377374, auc = 0.8950085, auc_precision_recall = 0.74177814, average_loss = 0.34238446, global_step = 11000, label/mean = 0.23622628, loss = 10.951594, prediction/mean = 0.23922005
